In [ ]:
import os


with open('participants.txt') as f:
    # peoples = f.readlines().remove('\n')
    peoples = f.read().splitlines()

from tqdm import tqdm

# # Loop through each email and extract the username part (before '@lehigh.edu')
# for people in peoples: using tqdm to show the progress bar
for people in tqdm(peoples):
    
    # Construct the GitLab repo URL using the email username
    repo_url = f"http://gitlab.cse.lehigh.edu/{people}-cse262/quiz-1"
    
    # Call the Python script to check the repo
    # You would use os.system() or subprocess to execute the shell command

    # if exception occurs, print the error message
    error = os.system(f"python check_repo.py {repo_url}")
    if error:
        print(f"Repo for {people} is not good!")
        print(error)
        continue
    else:
        # print(f"Repo for {people} is good!")
        print(error)

In [ ]:

# Path to the base directory where all the student folders are located
base_dir = os.path.abspath("/home/soz223/cse262grading/repo/quiz-1")  # Using absolute path to avoid relative path issues

result_file_path = os.path.join(base_dir, "results.txt")

# Ensure the results file exists
if not os.path.exists(result_file_path):
    with open(result_file_path, "w") as f:
        f.write("")  # Create the file if it doesn't exist

# The standard answer for the quiz
standard_answer = """
# CSE262 - Programming Languages - Fall 2024

# Quiz 1

Due: **9/9/2024 EOD**

Make at least one commit per question. Write your answers in this file.

## Question 1

List all the programming languages you have used before, in order of preference. Comment on what you like/dislike about each.

1. Python
   - Like: Easy syntax, rich libraries.
   - Dislike: Slower performance, dynamic typing issues.
2. Rust
   - Like: Memory safety, performance.
   - Dislike: Steep learning curve, strict compiler rules.
...
"""

# Function to grade a student's response using OpenAI API
def grade_response(student_answer, standard_answer):
    messages = [
        {"role": "system", "content": "You are a grading assistant for Rust programming language assignments."},
        {"role": "user", "content": f"Standard Answer:\n{standard_answer}"},
        {"role": "user", "content": f"Student's Response:\n{student_answer}"},
        {"role": "user", "content": (
            "Please grade the student's response based on the following criteria:\n"
            "1. If the answer is mostly correct and covers all major points, give 100 points.\n"
            "2. If the answer is somewhat correct but incomplete or has minor mistakes, give 80-90 points.\n"
            "3. If the answer is given but mostly incorrect or off-topic, give 60-70 points.\n"
            "4. If the answer is blank or completely irrelevant, give 0 points.\n"
            "Be generous, give 100 to most people. For scores less than 100, explain why the points were deducted. "
            "Provide the score and the reason in one line, reason must be specific, formatted as: 'Score: Reason for deduction'."
        )}
    ]
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=50,  # Allow more tokens to include the reason
        temperature=0.0,  # Use a lower temperature for more deterministic responses
        n=1,
        stop=None
    )
    
    # Extracting the score and reason
    result = response['choices'][0]['message']['content'].strip()
    return result

# Main function to loop through student directories and grade their quizzes
def main():
    # Example student folders list; this should be updated to the actual list of students
    # student_folders = ["student1", "student2", "student3"]
    # student_folders = ["student1", "student2", "student3"]
    student_folders = peoples

    # Array to track whether each student has been tested
    is_test_array = [False] * len(student_folders)

    # Iterate through each student folder
    for idx, student_folder in enumerate(student_folders):
        # Construct the path to the quiz-1 directory
        hw1_folder = os.path.normpath(os.path.join(base_dir, student_folder + '-cse262', 'quiz-1'))

        # Check if quiz-1 directory exists
        if os.path.exists(hw1_folder):
            # Path to README.md file
            readme_file_path = os.path.join(hw1_folder, "README.md")

            if os.path.exists(readme_file_path):
                # Read the student's response from README.md
                with open(readme_file_path, "r") as file:
                    student_answer = file.read()

                # Grade the student's response
                result = grade_response(student_answer, standard_answer)

                # Record the grade and reason in results.txt
                with open(result_file_path, "a") as f:
                    f.write(f"{student_folder}: {result}\n")

                print(f"Graded {student_folder}: {result}")
                is_test_array[idx] = True
            else:
                print(f"{student_folder} does not have a README.md file in the quiz-1 folder.")
                with open(result_file_path, "a") as f:
                    f.write(f"{student_folder} does not have a README.md file in the quiz-1 folder.\n")
        else:
            print(f"{student_folder} does not have a quiz-1 folder.")
            with open(result_file_path, "a") as f:
                f.write(f"{student_folder} does not have a quiz-1 folder.\n")

if __name__ == "__main__":
    main()